In [ ]:
!pip install pyarrow

In [ ]:
import pandas as pd
import pyarrow.parquet as pq

In [5]:
data = pd.read_parquet('/content/0000.parquet')

In [6]:
data.head()

,issue_area,issue_category,issue_sub_category,issue_category_sub_category,customer_sentiment,product_category,product_sub_category,issue_complexity,agent_experience_level,agent_experience_level_desc,conversation
0,Login and Account,Mobile Number and Email Verification,Verification requirement for mobile number or ...,Mobile Number and Email Verification -> Verifi...,neutral,Appliances,Oven Toaster Grills (OTG),medium,junior,"handles customer inquiries independently, poss...",Agent: Thank you for calling BrownBox Customer...
1,Cancellations and returns,Pickup and Shipping,Reasons for being asked to ship the item,Pickup and Shipping -> Reasons for being asked...,neutral,Electronics,Computer Monitor,less,junior,"handles customer inquiries independently, poss...",Agent: Thank you for calling BrownBox customer...
2,Cancellations and returns,Replacement and Return Process,Inability to click the 'Cancel' button,Replacement and Return Process -> Inability to...,neutral,Appliances,Juicer/Mixer/Grinder,medium,experienced,"confidently handles complex customer issues, e...",Agent: Thank you for calling BrownBox Customer...
3,Login and Account,Login Issues and Error Messages,Error message regarding exceeded attempts to e...,Login Issues and Error Messages -> Error messa...,neutral,Appliances,Water Purifier,less,inexperienced,"may struggle with ambiguous queries, rely on c...","Customer: Hi, I am facing an issue while loggi..."
4,Order,Order Delivery Issues,Delivery not attempted again,Order Delivery Issues -> Delivery not attempte...,negative,Electronics,Bp Monitor,medium,experienced,"confidently handles complex customer issues, e...",Agent: Thank you for contacting BrownBox custo...


In [ ]:

!pip install transformers
from transformers import pipeline

# Load the sentiment analysis pipeline
classifier = pipeline("sentiment-analysis")

# Assuming 'data' DataFrame has a column named 'text' containing the text for analysis
# Replace 'text' with the actual column name if it's different

# Process the first 10 rows for demonstration.
for index, row in data.head(10).iterrows():
  text = row['issue_area']
  try:
      result = classifier(text)
      print(f"Text: {text[:50]}...") # Print first 50 characters of text
      print(f"Sentiment: {result[0]['label']}, Score: {result[0]['score']}")
  except Exception as e:
    print(f"Error analyzing text '{text[:50]}...': {e}")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


Text: Login and Account...
Sentiment: NEGATIVE, Score: 0.9505677223205566
Text: Cancellations and returns...
Sentiment: NEGATIVE, Score: 0.9977436065673828
Text: Cancellations and returns...
Sentiment: NEGATIVE, Score: 0.9977436065673828
Text: Login and Account...
Sentiment: NEGATIVE, Score: 0.9505677223205566
Text: Order...
Sentiment: POSITIVE, Score: 0.9820718169212341
Text: Login and Account...
Sentiment: NEGATIVE, Score: 0.9505677223205566
Text: Cancellations and returns...
Sentiment: NEGATIVE, Score: 0.9977436065673828
Text: Shopping...
Sentiment: POSITIVE, Score: 0.9981168508529663
Text: Order...
Sentiment: POSITIVE, Score: 0.9820718169212341
Text: Login and Account...
Sentiment: NEGATIVE, Score: 0.9505677223205566


In [19]:
from textblob import TextBlob

# Function to determine sentiment
def analyze_sentiment(text):
    sentiment = TextBlob(text).sentiment.polarity
    if sentiment > 0:
        return 'positive'
    elif sentiment < 0:
        return 'negative'
    else:
        return 'neutral'

# Apply sentiment analysis to the 'conversation' column
data['predicted_sentiment'] = data['conversation'].apply(analyze_sentiment)

# Display the results
output = data[['conversation', 'customer_sentiment', 'predicted_sentiment']]
print(output)


                                          conversation customer_sentiment  \
0    agent: thank you for calling brownbox customer...            neutral   
1    agent: thank you for calling brownbox customer...            neutral   
2    agent: thank you for calling brownbox customer...            neutral   
3    customer: hi, i am facing an issue while loggi...            neutral   
4    agent: thank you for contacting brownbox custo...           negative   
..                                                 ...                ...   
995  agent: thank you for calling brownbox customer...           negative   
996  customer: hi, i received an email from brownbo...            neutral   
997  agent: thank you for calling brownbox customer...           negative   
998  customer: hi, i would like to check the status...            neutral   
999  customer: hi, i am calling because i have not ...           negative   

    predicted_sentiment  
0              positive  
1              positive

In [21]:
pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 4.9 MB/s eta 0:00:00


In [26]:
#CUSTOM KEYWORD BASED SENTIMENT
# Define positive and negative word lists
positive_words = ['thank', 'great', 'good', 'excellent', 'happy', 'amazing']
negative_words = ['issue', 'problem', 'bad', 'poor', 'angry', 'not']

def keyword_sentiment(text):
    text = str(text).lower()
    pos_count = sum(word in text for word in positive_words)
    neg_count = sum(word in text for word in negative_words)
    if pos_count > neg_count:
        return 'positive'
    elif neg_count > pos_count:
        return 'negative'
    else:
        return 'neutral'

data['keyword_sentiment'] = data['conversation'].apply(keyword_sentiment)

# Display the results
output = data[['conversation', 'customer_sentiment', 'keyword_sentiment']]
print(output.head())


                                        conversation customer_sentiment  \
0  agent: thank you for calling brownbox customer...            neutral   
1  agent: thank you for calling brownbox customer...            neutral   
2  agent: thank you for calling brownbox customer...            neutral   
3  customer: hi, i am facing an issue while loggi...            neutral   
4  agent: thank you for contacting brownbox custo...           negative   

  keyword_sentiment  
0          positive  
1          positive  
2          positive  
3          positive  
4          positive  


In [29]:
# NAIVE BAYES CLASSIFIER FOR SENTIMENT ANALYSIS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Prepare the data
X = data['conversation']
y = data['customer_sentiment']

# Convert text to numerical features using CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
X_vectorized = vectorizer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

# Train the Naive Bayes classifier
model = MultinomialNB()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

  frustrated       0.43      0.53      0.48        19
    negative       0.84      0.80      0.82        66
     neutral       0.84      0.90      0.87       107
    positive       0.00      0.00      0.00         8

    accuracy                           0.80       200
   macro avg       0.53      0.56      0.54       200
weighted avg       0.77      0.80      0.78       200



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [32]:
# Aspect-Based Sentiment Analysis (Rule-Based)
import re

# Define aspect-specific keywords
aspect_keywords = {
    'agent': ['agent', 'support', 'customer service'],
    'product': ['product', 'item', 'quality', 'defect'],
    'delivery': ['delivery', 'shipping', 'time', 'delayed']
}

# Define positive and negative keywords
positive_words = ['thank', 'great', 'good', 'excellent', 'happy', 'amazing']
negative_words = ['issue', 'problem', 'bad', 'poor', 'angry', 'not']

# Function for aspect-based sentiment analysis
def aspect_sentiment_analysis(text):
    text = str(text).lower()
    results = {}
    for aspect, keywords in aspect_keywords.items():
        if any(re.search(rf'\b{keyword}\b', text) for keyword in keywords):
            pos_count = sum(word in text for word in positive_words)
            neg_count = sum(word in text for word in negative_words)
            if pos_count > neg_count:
                results[aspect] = 'positive'
            elif neg_count > pos_count:
                results[aspect] = 'negative'
            else:
                results[aspect] = 'neutral'
    return results

# Apply the function
data['aspect_sentiment'] = data['conversation'].apply(aspect_sentiment_analysis)
print(data[['conversation', 'aspect_sentiment']].head())


                                        conversation  \
0  agent: thank you for calling brownbox customer...   
1  agent: thank you for calling brownbox customer...   
2  agent: thank you for calling brownbox customer...   
3  customer: hi, i am facing an issue while loggi...   
4  agent: thank you for contacting brownbox custo...   

                                    aspect_sentiment  
0                              {'agent': 'positive'}  
1  {'agent': 'positive', 'product': 'positive', '...  
2  {'agent': 'positive', 'product': 'positive', '...  
3                              {'agent': 'positive'}  
4      {'agent': 'positive', 'delivery': 'positive'}  


In [37]:
# Clustering-Based Sentiment Analysis
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert conversations to TF-IDF features
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(data['conversation'])

# Apply k-means clustering
num_clusters = 3  # For positive, neutral, negative
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
data['cluster'] = kmeans.fit_predict(X_tfidf)

# Manually label clusters (based on inspection)
cluster_labels = {0: 'positive', 1: 'neutral', 2: 'negative'}
data['cluster_sentiment'] = data['cluster'].map(cluster_labels)

# Display results
print(data[['conversation', 'cluster_sentiment']].head())


                                        conversation cluster_sentiment
0  agent: thank you for calling brownbox customer...          positive
1  agent: thank you for calling brownbox customer...          negative
2  agent: thank you for calling brownbox customer...          negative
3  customer: hi, i am facing an issue while loggi...          positive
4  agent: thank you for contacting brownbox custo...           neutral


In [39]:
# Optional(only if emojis are present in the data)
import re

def emoji_sentiment(text):
    positive_emojis = [r':\)', r':D', r'😊', r'😍', r'👍']
    negative_emojis = [r':\(', r':/', r'😠', r'😢', r'👎']

    if any(re.search(emoji, text) for emoji in positive_emojis):
        return 'positive'
    elif any(re.search(emoji, text) for emoji in negative_emojis):
        return 'negative'
    else:
        return 'neutral'

data['emoji_sentiment'] = data['conversation'].apply(emoji_sentiment)

# Display results
print(data[['conversation', 'emoji_sentiment']].head())


                                        conversation emoji_sentiment
0  agent: thank you for calling brownbox customer...         neutral
1  agent: thank you for calling brownbox customer...         neutral
2  agent: thank you for calling brownbox customer...         neutral
3  customer: hi, i am facing an issue while loggi...         neutral
4  agent: thank you for contacting brownbox custo...         neutral
